## Journal 04 Further Partitioning

In this journal we furtehr investigate the partitioning method we investigated in journal 2. The goal of this journal is try an discover what properties of a graph are associated with a particular partition leading to a greater profit versus another. We will start with the example from journal 02. 

In [1]:
import networkx as nx
import numpy as np
from numpy.linalg import inv, cond,norm
import numpy.linalg as lin 
from scipy.linalg import svdvals 
import helperfunctions.util
a = 6
c = 4
n = 20
G = np.zeros((20,20))
G[0,1:5] = 1
G[1,0] = G[1,5:8] = 1
G[2,3] = G[2,7] = G[2,8] = G[2,9] = 1
G[3,2] = G[3,4] = G[3,10] = G[3,11] = 1
G[4, 3] = G[4, 5] = G[4,12] = G[4,13] = 1
G[5, 4] = G[5,6] = G[5,14] = G[5,15] = 1
G[6, 5] = G[6,7] = G[6,16] = G[6,17] = 1
G[7,2] = G[7,6] = G[7,18] = G[7,19] = 1 

ones = np.ones((20,))
I = np.eye(20,20)
e = np.zeros((20,1))
f = np.zeros((20,1))
g = np.zeros((20,1))

for i in range(20):
    if( i < 2 ):
        e[i] = 1
    elif(i < 8):
        f[i] = 1
    else:
        g[i] = 1


We will wrap some parts into a function to make it easier to change other graphs. For this journal just for ease of use we will stick to this 20 node graph and only change the edges. 

In [2]:
def setup(G):
    """_summary_

    Args:
        G (Matrix): adj graph of matrix to set up for

    Returns:
        Mat, vec, rho: returns the constraint matrix and vector, and rho needed to solved reduced systems
    """
    rho = 0.95 / norm(G + G.T, ord=2)
    B = 0.5 * lin.inv( np.eye(20,20) - 2*rho * G)
    constraint_mat = np.zeros((3,3))
    constraint_vec = np.zeros((3,))

    constraint_mat[0,0] = 2 * e.T @ B @ e 
    constraint_mat[0,1] = e.T @ B @ f + f.T @ B @ e  
    constraint_mat[0,2] = e.T @ B @ g + g.T @ B @ e
    constraint_mat[1,0] = f.T @ B @ e  + e.T @ B @ f 
    constraint_mat[1,1] = 2 * f.T @ B @ f
    constraint_mat[1,2] = f.T @ B @ g + g.T @ B @ f 
    constraint_mat[2,0] = e.T @ B @ g + g.T @  B @ e 
    constraint_mat[2,1] = f.T @ B @ g + g.T @ B @ f
    constraint_mat[2,2] = 2 * g.T @ B @ g
    constraint_vec[0,] = a * e.T @ B @ ones + c * ones.T @ B @ e 
    constraint_vec[1,] = a * f.T @ B @ ones + c * ones.T @ B @ f 
    constraint_vec[2,] = a * g.T @ B @ ones + c * ones.T @ B @ g
    return constraint_mat, constraint_vec, rho



In [3]:
[constraint_mat_g, constraint_vec_g, rho_g] = setup(G)

Recall we had three partitions

In [4]:
part_pq = np.array([[1,0],[1,0], [0,1]])
part_pr = np.array([[1,0],[0,1], [1,0]])
part_qr = np.array([[1,0],[0,1],[0,1]])
# new constraints 
v_pq = inv(part_pq.T @ constraint_mat_g @ part_pq) @ (part_pq.T @ constraint_vec_g)
v_pr = inv(part_pr.T @ constraint_mat_g @ part_pr) @ (part_pr.T @ constraint_vec_g)
v_qr = inv(part_qr.T @ constraint_mat_g @ part_qr) @ (part_qr.T @ constraint_vec_g)

With optimal prices given by

In [5]:
price_pq_g = np.zeros((20,))
price_pr_g = np.zeros((20,))
price_qr_g = np.zeros((20,))
for i in range(20):
    if(i < 2):
        price_pq_g[i] = v_pq[0]
        price_pr_g[i] = v_pr[0]
        price_qr_g[i] = v_qr[0]
    elif(i < 8):
        price_pq_g[i] = v_pq[0]
        price_pr_g[i] = v_pr[1]
        price_qr_g[i] = v_qr[1] 
    else:
        price_pq_g[i] = v_pq[1]
        price_pr_g[i] = v_pr[0]
        price_qr_g[i] = v_qr[1]

true_vector_g = helperfunctions.util.price_vector(a, c, rho_g, G)
true_profit = helperfunctions.util.optProfit(G, rho_g, a, c)


Which leads to profits of 

In [6]:
profit_pq = helperfunctions.util.computeProfit(G, price_pq_g, rho_g, a, c)
profit_pr = helperfunctions.util.computeProfit(G, price_pr_g, rho_g, a, c)
profit_qr = helperfunctions.util.computeProfit(G, price_qr_g, rho_g, a, c)
print("Profit_pq", profit_pq)
print("Profit_pr", profit_pr)
print("Profit_qr", profit_qr)
print("Compared to the optimal of", true_profit)

Profit_pq 53.64776127206805
Profit_pr 36.441155376588014
Profit_qr 117.93024338508046
Compared to the optimal of 129.40991805074188


In this case the ordering of performance for different partitions was 1. qr 2. pq 3. pr and there is a notable gap between the three classes. Consider now a new system from graph H

In [7]:
H = np.zeros((20,20))
H[0,1:5] = 1
H[1,0] = H[1,5:8] = 1
H[2,3] = H[2,7] = H[2,8] = H[2,9] = 1
H[3,2] = H[3,4] = H[3,10] = H[3,11] = 1
H[4, 3] = H[4, 5] = H[4,12] = H[4,13] = 1
H[5, 4] = H[5,6] = H[5,14] = H[5,15] = 1
H[6, 5] = H[6,7] = H[6,16] = H[6,17] = 1
H[7,2] = H[7,6] = H[7,18] = H[7,19] = 1 
H[19,7] = H[18,7] = H[17,6] = H[16,6] = 1
H[15,5] = H[14,5] = H[13,4] = H[12,4] = 1
H[11,3] = H[10,3] = H[9, 2] = H[8,2] = 1 
H[19,0] = H[18,1] = H[17,0] = H[16,1] = 1
H[15,0] = H[14,1] = H[13,0] = H[12,1] = 1
H[11,0] = H[10,1] = H[9,0] = H[8,1] = 1
constraint_mat_h, constraint_vec_h, rho_h = setup(H)

From which can play the same profit finding game

In [8]:
v_pq = inv(part_pq.T @ constraint_mat_h @ part_pq) @ (part_pq.T @ constraint_vec_h)
v_pr = inv(part_pr.T @ constraint_mat_h @ part_pr) @ (part_pr.T @ constraint_vec_h)
v_qr = inv(part_qr.T @ constraint_mat_h @ part_qr) @ (part_qr.T @ constraint_vec_h)
price_pq_h = np.zeros((20,))
price_pr_h = np.zeros((20,))
price_qr_h = np.zeros((20,))
for i in range(20):
    if(i < 2):
        price_pq_h[i] = v_pq[0]
        price_pr_h[i] = v_pr[0]
        price_qr_h[i] = v_qr[0]
    elif(i < 8):
        price_pq_h[i] = v_pq[0]
        price_pr_h[i] = v_pr[1]
        price_qr_h[i] = v_qr[1] 
    else:
        price_pq_h[i] = v_pq[1]
        price_pr_h[i] = v_pr[0]
        price_qr_h[i] = v_qr[1]


profit_pq = helperfunctions.util.computeProfit(H, price_pq_h, rho_h, a, c)
profit_pr = helperfunctions.util.computeProfit(H, price_pr_h, rho_h, a, c)
profit_qr = helperfunctions.util.computeProfit(H, price_qr_h, rho_h, a, c)
true_profit = helperfunctions.util.optProfit(H, rho_h, a, c)
true_vector_h = helperfunctions.util.price_vector(a, c, rho_h, H)

print("Profit_pq", profit_pq)
print("Profit_pr", profit_pr)
print("Profit_qr", profit_qr)
print("Compared to the optimal of", true_profit)

Profit_pq 157.2143828281001
Profit_pr 163.40591196034893
Profit_qr 106.56771460475835
Compared to the optimal of 170.9657981411967


In this case the ordering is 1. pr 2. pq, 3. qr. So the question is what about G and H indicates which system we should consider. It isn't entirely obvious so we will investigate some natural ideas. 

## Guess 1: condensed matrix norms
Where we will look at if the order $\| C B C^T \|$ predicts the order of profits where B is the matrix $(I - 2 \rho G)^{-1}$

In [9]:
h_norms_2 = [0,0,0]
h_norms_2[0] = norm(part_pq.T @ constraint_mat_h @ part_pq, ord="fro")
h_norms_2[1] = norm(part_pr.T @ constraint_mat_h @ part_pr, ord="fro")
h_norms_2[2] = norm(part_qr.T @ constraint_mat_h @ part_qr, ord="fro")
print(h_norms_2)


[109.6129190212143, 108.1757980529781, 161.49954764110774]


In [10]:
g_norms_2 = [0,0,0]
g_norms_2[0] = norm(part_pq.T @ constraint_mat_g @ part_pq, ord="fro")
g_norms_2[1] = norm(part_pr.T @ constraint_mat_g @ part_pr, ord="fro")
g_norms_2[2] = norm(part_qr.T @ constraint_mat_g @ part_qr, ord="fro")
print(g_norms_2)

[41.84969521216497, 37.23512531350235, 50.819925076471286]


This shows that nrom as stated is not the correct approach because both systems are ordered via norm but have different prerferrences for partitions. 

## Idea 2, "distance" from orginal contraint
Another guess is if we have some partition $C$ and squish the constraint to $C^T A C$ and unsquish it by averaging the partition that induces the least distance is best. 

### Example 
if $A = \begin{bmatrix} 1 & 2 & 3 \\ 4 & 5 & 6 \\ 7 & 8 & 9 \end{bmatrix}, C = \begin{bmatrix} 1 & 0 \\ 1 & 0 \\ 0 & 1 \end{bmatrix}$ then we have $C^T A C = \begin{bmatrix} 1+2+4+5 & 3 + 6 \\ 7 + 8 & 9 \end{bmatrix} = \begin{bmatrix} 12 & 9 \\ 15 & 9\end{bmatrix}$. But if we want to map back to a 3x3 matrix we lose some information and instead have 
$guess_C = \begin{bmatrix} 3 & 4.5 \\ 7.5 & 4.5 \end{bmatrix}$. I am proposing that the distance between this averaged matrix and the original matrix matters

This shows that such a 

In [11]:
print(part_pq)
C = np.array([[1,2,3], [4,5,6], [7,8,9]])
print(C)
print(part_pq.T @ C @ part_pq)

[[1 0]
 [1 0]
 [0 1]]
[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[12  9]
 [15  9]]


The number of elements that are merged to together is the number of ones in column i * # ones in row j of the partition. 

In [12]:
rev_pq = np.array([[0.5, 0.5, 0], [0, 0.0, 1]])
rev_qr = np.array([ [1, 0, 0], [0, 0.5, 0.5]])
rev_pr = np.array([ [0.5, 0, 0.5], [0, 1, 0]])


To make this averaging operation epxlicit we will show the operations on the matrix C

In [13]:
print("C orginal")
print(C)
print("C condensed in PQ fashion")
c_pq = part_pq.T @ C @ part_pq
print(c_pq)
print("Averaged back to 3x3")
c_pq_avg =  rev_pq.T @ c_pq @ rev_pq
print(c_pq_avg)
print("C condense in QR fashion")
c_qr = part_qr.T @  C @ part_qr
print(c_qr)
print("Averaged back to 3x3")
c_qr_avg = rev_qr.T @ c_qr @ rev_qr
print(c_qr_avg)
print("C condensed in PR fashion")
c_pr = part_pr.T @ C @ part_pr
print(c_pr)
print("Averaged back to 3x3")
c_pr_avg = rev_pr.T @ c_pr @ rev_pr
print(c_pr_avg)

C orginal
[[1 2 3]
 [4 5 6]
 [7 8 9]]
C condensed in PQ fashion
[[12  9]
 [15  9]]
Averaged back to 3x3
[[3.  3.  4.5]
 [3.  3.  4.5]
 [7.5 7.5 9. ]]
C condense in QR fashion
[[ 1  5]
 [11 28]]
Averaged back to 3x3
[[1.  2.5 2.5]
 [5.5 7.  7. ]
 [5.5 7.  7. ]]
C condensed in PR fashion
[[20 10]
 [10  5]]
Averaged back to 3x3
[[5. 5. 5.]
 [5. 5. 5.]
 [5. 5. 5.]]


Now we calculatet this for g and h

In [14]:
g_pq = part_pq.T @ constraint_mat_g @ part_pq
h_pq = part_pq.T @ constraint_mat_h @ part_pq
g_pr = part_pr.T @ constraint_mat_g  @ part_pr
h_pr = part_pr.T @ constraint_mat_h @ part_pr
g_qr = part_qr.T @ constraint_mat_g @ part_qr
h_qr = part_qr.T @ constraint_mat_h @ part_qr

g_pq_avg = rev_pq.T @ g_pq @ rev_pq
h_pq_avg = rev_pq.T @ h_pq @ rev_pq
g_pr_avg = rev_pr.T @ g_pr @ rev_pr
h_pr_avg = rev_pr.T @ h_pr @ rev_pr
g_qr_avg = rev_qr.T @ g_qr @ rev_qr
h_qr_avg = rev_qr.T @ h_qr @ rev_qr

g_diffs = [norm(constraint_mat_g - g_pq_avg, ord="fro"),
           norm(constraint_mat_g - g_pr_avg, ord="fro"),
           norm(constraint_mat_g - g_qr_avg, ord="fro")
            ]
print(g_diffs)
h_diffs = [norm(constraint_mat_h - h_pq_avg, ord="fro"),
           norm(constraint_mat_h - h_pr_avg, ord="fro"),
           norm(constraint_mat_h - h_qr_avg, ord="fro")
            ]
h_diffs

[14.681831981677286, 7.276840447700115, 11.273696135698133]


[38.62692757794101, 35.834119140963786, 5.2917234007707705]

Sadly this also indicates this is not the answer.

Yet another guess is that it is the distance from the original price vectors that matter

In [15]:
true_vector_g = np.reshape(true_vector_g, (20,))
true_vector_h = np.reshape(true_vector_h, (20,))

g_vec_diffs = [ sum([ abs(i) for i in true_vector_g - price_pq_g]),
                sum([ abs(i) for i in true_vector_g - price_pr_g]),
                sum([ abs(i) for i in true_vector_g - price_qr_g])]

g_vec_diffs

h_vec_diffs = [ sum([ abs(i) for i in true_vector_h - price_pq_h]),
                sum([ abs(i) for i in true_vector_h - price_pr_h]),
                sum([ abs(i) for i in true_vector_h - price_qr_h])]
print(h_vec_diffs)

[16.56516748914526, 16.326147028781243, 70.00053210668099]


In [16]:
I = np.zeros((20,20))
I[0,1:5] = 1
I[1,0] = I[1,5:8] = 1
#I[2,3] = I[2,7] = I[2,8] = I[2,9] = 1
#I[3,2] = I[3,4] = I[3,10] = I[3,11] = 1
#I[4, 3] = I[4, 5] = I[4,12] = I[4,13] = 1
#I[5, 4] = I[5,6] = I[5,14] = I[5,15] = 1
#I[6, 5] = I[6,7] = I[6,16] = I[6,17] = 1
#I[7,2] = I[7,6] = I[7,18] = I[7,19] = 1 
I[19,7] = I[18,7] = I[17,6] = I[16,6] = 1
I[15,5] = I[14,5] = I[13,4] = I[12,4] = 1
I[11,3] = I[10,3] = I[9, 2] = I[8,2] = 1 
I[19,0] = I[18,1] = I[17,0] = I[16,1] = 1
I[15,0] = I[14,1] = I[13,0] = I[12,1] = 1
I[11,0] = I[10,1] = I[9,0] = I[8,1] = 1
constraint_mat_i, constraint_vec_i, rho_i = setup(I)
v_pq = inv(part_pq.T @ constraint_mat_i @ part_pq) @ (part_pq.T @ constraint_vec_i)
v_pr = inv(part_pr.T @ constraint_mat_i @ part_pr) @ (part_pr.T @ constraint_vec_i)
v_qr = inv(part_qr.T @ constraint_mat_i @ part_qr) @ (part_qr.T @ constraint_vec_i)
price_pq_i = np.zeros((20,))
price_pr_i = np.zeros((20,))
price_qr_i = np.zeros((20,))
for i in range(20):
    if(i < 2):
        price_pq_i[i] = v_pq[0]
        price_pr_i[i] = v_pr[0]
        price_qr_i[i] = v_qr[0]
    elif(i < 8):
        price_pq_i[i] = v_pq[0]
        price_pr_i[i] = v_pr[1]
        price_qr_i[i] = v_qr[1] 
    else:
        price_pq_i[i] = v_pq[1]
        price_pr_i[i] = v_pr[0]
        price_qr_i[i] = v_qr[1]


profit_pq = helperfunctions.util.computeProfit(I, price_pq_i, rho_i, a, c)
profit_pr = helperfunctions.util.computeProfit(I, price_pr_i, rho_i, a, c)
profit_qr = helperfunctions.util.computeProfit(I, price_qr_i, rho_i, a, c)
true_profit = helperfunctions.util.optProfit(I, rho_i, a, c)
true_vector_i = helperfunctions.util.price_vector(a, c, rho_i, I)
true_vector_i = np.reshape(true_vector_i, (20,))
print("Profit_pq", profit_pq)
print("Profit_pr", profit_pr)
print("Profit_qr", profit_qr)
print("Compared to the optimal of", true_profit)

Profit_pq 127.40277348762262
Profit_pr 64.51458537683735
Profit_qr 25.51727683323866
Compared to the optimal of 152.82211853097766


In [17]:
i_vec_diffs = [ sum([ abs(i) for i in true_vector_i - price_pq_i]),
                sum([ abs(i) for i in true_vector_i - price_pr_i]),
                sum([ abs(i) for i in true_vector_i - price_qr_i])]
print(i_vec_diffs)

[49.042566770804186, 157.1712302186006, 215.94864517296023]


Althought it seems like we have found a solution but caution is needed. It is not the case that the vector closest in distance to the real price vector is best. 
Consider the following example where we compare the profit generated and distance from the optimal price. 

In [40]:
print("true price vector")
print(true_vector_i)
print("Price with pq partitioning")
print(price_pq_i)
price_test = np.zeros((20,))
avg_1 = sum(true_vector_i[:8])/8
avg_2 = sum(true_vector_i[8:])/12
price_test[:8] = avg_1
price_test[8:] = avg_2
print("closest vector where all the nodess in P or Q are equal and all the nodes in R are the same")
print(price_test)


true price vector
[-0.98874332 -0.98874332 -7.87195124 -7.87195124 -7.87195124 -7.87195124
 -7.87195124 -7.87195124 15.87570347 15.87570347 15.87570347 15.87570347
 15.87570347 15.87570347 15.87570347 15.87570347 15.87570347 15.87570347
 15.87570347 15.87570347]
Price with pq partitioning
[-3.76977522 -3.76977522 -3.76977522 -3.76977522 -3.76977522 -3.76977522
 -3.76977522 -3.76977522 14.30341624 14.30341624 14.30341624 14.30341624
 14.30341624 14.30341624 14.30341624 14.30341624 14.30341624 14.30341624
 14.30341624 14.30341624]
closest vector where all the nodess in P or Q are equal and all the nodes in R are the same
[-6.15114926 -6.15114926 -6.15114926 -6.15114926 -6.15114926 -6.15114926
 -6.15114926 -6.15114926 15.87570347 15.87570347 15.87570347 15.87570347
 15.87570347 15.87570347 15.87570347 15.87570347 15.87570347 15.87570347
 15.87570347 15.87570347]


In [43]:
helperfunctions.util.computeProfit(I, price_test, rho_i, a, c)
print("price test distance from true price", norm(true_vector_i - price_test))
print("optimal pq price distance from true price",  norm(true_vector_i - price_pq_i))

price test distance from true price 8.430173603209322
optimal pq price distance from true price 12.087200318095782


##  Conclusion
Sadly we end this journal somewhat confused about how to progress in terms of how to progress to an answer of what partition is best. It seems like a reasonable guess that the one that induces the smallest dstiance to the true vector is best but we reiterate this is not the same as just picking the vector itself that minimizes the distance.